In [1]:
import os 
import rasterio
import geopandas as gpd
import numpy as np

In [2]:
path =  "/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth"
years = list(range(2014,2021))
cutline_path = "/home/jovyan/work/satellite_data/ku_sync/South_Africa/cutlines/SA_all.geojson"

In [3]:
df = gpd.read_file(cutline_path)

In [14]:
for y in years:
    df_year = df[df["year"] == str(y)]
    p = os.path.join(path,str(y))
    arr_all = df_year.y_idx.values.astype("int")
    if os.path.isdir(p):
        idxs = [int(i.split("_")[-1].split(".")[0]) for i in os.listdir(p) if i.endswith(".tif")]
        idxs = sorted(idxs)
        arr_avail = np.array(idxs)
    else:
        arr_avail = np.array([])
    missing_idxs = np.setdiff1d(arr_all,arr_avail)
    unique_elements, counts = np.unique(arr_avail, return_counts=True)
    duplicate_elements = unique_elements[counts > 1]
    print(f"Year {y}: {missing_idxs.tolist()}")
    print(f"Duplicate tiles: {duplicate_elements.tolist()}  \n #############")

Year 2014: []
Duplicate tiles: [101, 109, 201]  
 #############
Year 2015: []
Duplicate tiles: [9, 64, 83, 101, 201]  
 #############
Year 2016: []
Duplicate tiles: [1, 101, 104, 105, 106, 181, 201, 243, 244, 245, 246, 301, 433, 434, 435, 436, 437, 439, 440, 441, 442, 443, 444, 445, 501, 601]  
 #############
Year 2017: []
Duplicate tiles: [2, 3, 101, 167, 168, 169, 177, 201, 292, 302, 305, 306, 307, 350, 351, 352, 353, 354, 355, 356, 357]  
 #############
Year 2018: []
Duplicate tiles: [11, 12, 71, 99, 134, 135, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 161, 179, 180, 181, 182, 183, 201]  
 #############
Year 2019: []
Duplicate tiles: [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]  
 #############
Year 2020: []
Duplicate tiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]  
 #############


# Check for corrupt tiles


In [25]:
for y in [2018]:#years:
    p = os.path.join(path,str(y))
    if os.path.isdir(p):
        tiles = [os.path.join(p,i) for i in os.listdir(p) if i.endswith(".tif")]
        
        for i in tiles:
            with rasterio.open(i) as src:
                try:
                    a = src.read(1, window=((0, 1), (0, 1)))
                    correct = True
                except:
                    correct = False
                    print(correct)
                    print(f"{os.path.basename(i)}")
                #print(a)

# Overlapping tiles

In [34]:
import rasterio
from shapely.geometry import box

path = "/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018"
f = "tmp_shape_13_03_2023_153432_315.tif"
files = [os.path.join(path,i) for i in os.listdir(path) if i.endswith(".tif") and i != f]

for i in files:
    with rasterio.open(os.path.join(path,"tmp_shape_13_03_2023_153432_315.tif")) as src1, rasterio.open(i) as src2:
        # Get the spatial extent of each file
        extent1 = box(*src1.bounds)
        extent2 = box(*src2.bounds)

        # Check if the two extents overlap
        if extent1.intersects(extent2):
            print(i)

/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018/tmp_shape_15_03_2023_135759_284.tif
/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018/tmp_shape_13_03_2023_153432_312.tif
/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018/tmp_shape_13_03_2023_150953_229.tif
/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018/tmp_shape_15_03_2023_001614_249.tif
/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018/tmp_shape_15_03_2023_001614_255.tif
/home/jovyan/work/satellite_data/tmp/inference/smp_unet_mitb3_08_03_2023_170715.pth/2018/2018_330.tif
